In [45]:
import scipy

In [1]:
import tqdm
from tqdm import tqdm
import pandas as pd 
import random
import spacy

df = pd.read_excel('../Excel_files/Full_Poem_Dataset_12-17.xlsx')
df = df.drop(columns=['Unnamed: 0'])
records = df.to_dict('records')
random.choice(records)

{'Text': 'В этом городе серозубом\nНе осталось дней и ночей,\nТолько ангелы дуют в трубы,\nРудименты былых печей.\n \nНа картине тень от картины,\nНа кровати память страстей,\nПод гнездом шершавым осиным\nСпит газета без новостей.\n \nУ порога замерли грабли,\nРотозея вечный тотем.\nВ ржавой мойке застыли капли,\nДо истоков не долетев.\n \nРаз в бессмертье стражник сторукий\nПромелькнёт на бледном коне,\nОглушит позабытым звуком\nИ исчезнет в пустом окне.\n \nНе припомнить прошлого лета,\nНе сменять пятак по рублю,\nВ этом мире чёрствых багетов\nСкорость света равна нулю.',
 'Author': 'Эдуард Маркович',
 'Before or after': 'After',
 'Source': 'No War Poetry',
 'Date posted': 'None',
 'UniqueIndex': 1983}

In [56]:
df.to_json('Full_Poem_Dataset_2-3.json', orient='records', force_ascii=False)

### Statistics for Methods section

In [2]:
nlp = spacy.load("ru_core_news_lg")

In [3]:
for rec in tqdm(records):
    doc = nlp(rec['Text'])
    rec['doc'] = doc

100%|██████████████████████████████████████| 3222/3222 [05:08<00:00, 10.44it/s]


In [8]:
# look at lemma counts
tokenCt = dict()
poemCt = dict()
for p in ['Before','After']:
    tokenCt.setdefault(p, 0)
    poemCt.setdefault(p, 0)
for rec in tqdm(records):
    poemCt[rec['Before or after']] += 1
    for t in rec['doc']:
        tokenCt[rec['Before or after']] += 1 
print(tokenCt)
print(poemCt)

100%|███████████████████████████████████| 3222/3222 [00:00<00:00, 11634.37it/s]

{'Before': 291769, 'After': 266511}
{'Before': 1661, 'After': 1561}


#### Calculating two sample one sided t-test

In [53]:
lemmaCts = dict()
totalPoemCts = dict()
scipyArray = dict()

for r in tqdm(records):
    p = r['Before or after']
    totalPoemCts.setdefault(p, 0)
    lemmasFound = set()
    totalPoemCts[p] += 1
    for t in r['doc']:
        if t.lemma_ not in lemmasFound:
            lemmaCts.setdefault(t.lemma_, {'Before' : 0, 'After' : 0})
            lemmaCts[t.lemma_][p] += 1
            lemmasFound.add(t.lemma_)
        else:
            continue
lemmaFreqs = dict()
for word in lemmaCts:
    lemmaFreqs.setdefault(word, {'Before' : 0, 'After' : 0})
    for p in lemmaCts[word]:
        lemmaFreqs[word][p] = lemmaCts[word][p]/totalPoemCts[p]
        lemmaFreqs[word][p+'Ct'] = lemmaCts[word][p]
        
# create arrays for scipy
scipyArray = dict()
for lem in lemmaCts:
    scipyArray.setdefault(lem, {'Before' : [], 'After' : []})

for r in tqdm(records):
    p = r['Before or after']
    recLemmas = set([t.lemma_ for t in r['doc']])
    for rl in recLemmas:
        scipyArray[rl][p].append(1)
    for lem in lemmaCts:
        if lem not in recLemmas:
            scipyArray[lem][p].append(0)

100%|██████████████████████████████████████| 3222/3222 [03:36<00:00, 14.88it/s]


In [57]:
len(scipyArray['бог']['Before']), len(scipyArray['бог']['After'])

(1661, 1561)

In [58]:
lemmaFreqs['бог']

{'Before': 0.07405177603853101,
 'After': 0.1108263933376041,
 'BeforeCt': 123,
 'AfterCt': 173}

In [59]:
for keyword in tqdm(lemmaCts):
    if lemmaCts[keyword]['Before'] + lemmaCts[keyword]['After'] >= 25:
        a1 = scipyArray[keyword]['Before']
        a2 = scipyArray[keyword]['After']
        test = scipy.stats.ttest_ind(a1, a2, axis=0, equal_var=True, nan_policy='propagate', 
                              permutations=None, random_state=None, alternative='two-sided', trim=0)
        lemmaFreqs[keyword]['stats'] = {'statistic' : test.statistic, 'pvalue' : test.pvalue}

100%|█████████████████████████████████| 43565/43565 [00:02<00:00, 15383.09it/s]


In [62]:
sig = []
notsig = []
for keyword in lemmaCts:
    if lemmaCts[keyword]['Before'] + lemmaCts[keyword]['After'] >= 25:
        if lemmaFreqs[keyword]['stats']['pvalue'] < .05:
            sig.append(keyword)
        else:
            notsig.append(keyword)

In [66]:
with open(f'frequencyData2-3.json', 'w', encoding='utf-8') as f:
    json.dump(lemmaCts, f, ensure_ascii=False, indent=4)

NameError: name 'json' is not defined

#### Language Detection

In [2]:
# !pip3 install spacy-language-detection

In [3]:
import spacy
from spacy.language import Language

from spacy_language_detection import LanguageDetector


def get_lang_detector(nlp, name):
    return LanguageDetector(seed=42)  # We use the seed 42


In [4]:

nlp_model = spacy.load("ru_core_news_lg")
Language.factory("language_detector", func=get_lang_detector)
nlp_model.add_pipe('language_detector', last=True)

# Sentence level language detection
text = "This is English text. Er lebt mit seinen Eltern und seiner Schwester in Berlin. Yo me divierto todos los días en el parque. Je m'appelle Angélica Summer, j'ai 12 ans et je suis canadienne."
doc = nlp_model(text)
for i, sent in enumerate(doc.sents):
    print(sent, sent._.language)

This is English text. {'language': 'en', 'score': 0.9999987929307774}
Er lebt mit seinen Eltern und seiner Schwester in Berlin. {'language': 'de', 'score': 0.999996045846908}
Yo me divierto todos los días en el parque. {'language': 'es', 'score': 0.9999960751128256}
Je m'appelle Angélica Summer, j'ai 12 ans et je suis canadienne. {'language': 'fr', 'score': 0.9999960488878062}


In [ ]:
from tqdm import tqdm 
random.shuffle(records)
for rec in tqdm(records):
    rec['Doc'] = nlp_model(rec['Text'])

100%|███████████████████████████████████████| 3252/3252 [05:13<00:00, 10.36it/s]


In [105]:
langDict = dict()
for rec in tqdm(records):
    for sent in rec['Doc'].sents:
        langDict.setdefault(sent._.language['language'], [])
        langDict[sent._.language['language']].append((sent, rec))

100%|███████████████████████████████████████| 3252/3252 [07:47<00:00,  6.96it/s]


In [106]:
langDict.keys()

dict_keys(['ru', 'UNKNOWN', 'bg', 'mk', 'uk', 'en', 'et', 'hr', 'tl', 'ro', 'lt', 'de', 'id', 'ca', 'sk'])

In [146]:
lg = 'sk'
print(len(langDict[lg]))
[print('Unique index:',d[1]['UniqueIndex'],'\n', d[0],'\n--') for d in random.sample(langDict[lg],1)]

1
Unique index: 1652 
 Како добро живети…
…Ako žiť dobre…
…Kako dobro živeti. 
--


[None]

In [151]:
# 523
r = [r for r in records if r['UniqueIndex'] == 3239][0]

In [157]:
r['Source'], r['Author'], r['Date posted']

('essentialpoetry', 'Линор Горалик', datetime.datetime(2018, 5, 9, 0, 0))

In [153]:
print(r['Text'])

Вот красным лесом красная лиса, – 
а он лежит, смешавшись с автоматом, 
в осеннем красном буром чернозёме, 
неглубоко, – 
и вот лиса несётся, 
пересекая сердце, горло, сердце, – 
подскакивает, лапой влезши в душу, 
отряхивает лапу, мчится дальше, – 
И он кричит распавшейся гортанью: 
 
КАКОГО ХУЯ, ГОСПОДИ, – ЗА ЧТО?! 
 
Я не успел – я инвалид по зренью, – 
я не успел, – они меня в апреле, 
когда уже исход и всё понятно, 
когда таких, как я, – едва одетых, 
полуслепых, хромых или безусых, – 
от киндер, кирхе, запаха из кюхен, – 
в зелёный их апрельский красный лес, 
где я от крови ничего не видел, 
и красный зверь, и горло, сердце, горло – 
а я ни разу даже не пальнул, 
я не успел – 
какого хуя, Боже?! 
 
ТАК ДАЙ МНЕ, ДАЙ МНЕ, ДАЙ МНЕ ЧТО-НИБУДЬ!!! 
 
И тут лиса упала и лежит.


In [133]:
# other

In [2]:
total = 0
for rec in records:
    total += len(str(rec['Text']).split(' '))
# 345043
total

342235

In [5]:
df['Source'].value_counts()

essentialpoetry      945
No War Poetry        724
metajournal          662
ROAR V3              350
Facebook             253
ROAR V2              234
Personal Telegram     54
Name: Source, dtype: int64

In [4]:
df['Before or after'].value_counts()

Before    1661
After     1561
Name: Before or after, dtype: int64

### Remove duplicates from corpus

In [222]:
print(random.choice(records)['Text'][:200])

Развивая Бродского
1
Я хотел бы жить, Иосиф, как можно дольше, пока
ритмично дыханье и тянется на бумаге строка,
жизнь - гостиный двор, мы временные постояльцы,
и какое мне дело до того городка,
где и


In [223]:
longStringCts = dict()
string2Recs = dict()
string2RecIdx = dict()

windowSize = 100
for i, rec in tqdm(enumerate(records)):
    text = rec['Text']
    rec['UniqueIndex'] = i
    if isinstance(text, str):
        for j in range(len(text)-windowSize):
            windowText = text[j:j+windowSize]
            longStringCts.setdefault(windowText, 0)
            longStringCts[windowText] += 1
            string2Recs.setdefault(windowText, [])
            string2Recs[windowText].append(rec)
            # string2RecIdx.setdefault(windowText, set())
            # string2RecIdx[windowText].add(i)

3502it [00:08, 406.86it/s] 


In [224]:
relevantCts = dict()
for s in tqdm(longStringCts):
    if longStringCts[s] > 1:
        relevantCts[s] = longStringCts[s]

100%|████████████████████████████| 2284719/2284719 [00:01<00:00, 1186974.80it/s]


In [225]:
overlappingRecs = []
for s in tqdm(relevantCts):
    shareString = string2Recs[s]
    if shareString not in overlappingRecs:
        # ids = [r['UniqueIndex'] for r in shareString]
        # if set(ids) 
        overlappingRecs.append(shareString)
len(overlappingRecs)

100%|█████████████████████████████████| 64074/64074 [00:00<00:00, 133374.94it/s]


228

In [143]:
import datetime

In [226]:
def decideRec(recList):
    # if just one, return rec
    if len(recList) == 1:
        return recList[0:]
    # convert to datetimes
    for r in recList:
        if not isinstance(r['Date posted'], datetime.datetime):
            # essentialpoetry      1023
            # No War Poetry         793
            # metajournal           713
            # ROAR V3               357
            # Facebook              281
            # ROAR V2               269
            # Personal Telegram      66
            if r['Source'] == 'ROAR V2':
                r['Date posted'] = datetime.datetime(2022, 6, 24, 0, 0)
            elif r['Source'] == 'ROAR V3':
                r['Date posted'] = datetime.datetime(2022, 8, 24, 0, 0)
            else:
                r['Date posted'] = datetime.datetime(2022,12,17,0,0)
    # compare date
    sortedByDate = []
    for i, rec in enumerate(recList):
        sortedByDate.append((rec['Date posted'], i))
    sortedByDate = sorted(sortedByDate)
    # check types
    if isinstance(sortedByDate[0], datetime.datetime) and isinstance(sortedByDate[1], datetime.datetime):
        if sortedByDate[0] != sortedByDate[1]:
            return recList[sortedByDate[0][1]]
    else:
        idxs = [s[1] for s in sortedByDate[1:]]
        return [recList[j] for j in idxs]
    
    # compare length of text
    sortedByLen = []
    for i, rec in enumerate(recList):
        sortedByLen.append((len(rec['Text']), i))
    sortedByLen = sorted(sortedByLen)
    idxs = [s[1] for s in sortedByLen[1:]]
    return [recList[j] for j in idxs]

In [228]:
oR = random.choice(overlappingRecs)
print([i['UniqueIndex'] for i in oR])
decideRec(oR)

[162, 171, 175]


[{'Text': 'апофеоз войны\nсажи золы бурьяна\nвсе до одной черны\nклавиши фортепьяно\nбудем играть на нём\nразмазывая копоть\nпод проливным огнем\nв чьей-то крови по локоть\n+\nтрам-там-там\nтрам-там-там\nмаменькин сынок\nпомещён по частям\nв мусорный мешок\nмина взрыв\nмина взрыв\nиз отчёта стёрт\nГосударству — призыв,\nродине — аборт.\n+\nсветомаскировка\nночи звёздный час\nприлегла винтовка\nприкорнул фугас\nа когда проснутся\nпо команде пли\nзвёзды отвернутся\nот моей Земли\n+\nДело мое дрянь.\nСделана сказка пылью.\nПлáчу — плачу дань\nгневу, стыду, бессилью.\nСилой берут дев,\n“Мурку” орут орки.\nТы близорук, гнев.\nСлёзы, вы дальнозорки.\n+\nРадость передышки краткой,\nшуточки о женской доле\nпосле схватки, перед схваткой\nв мариупольском роддоме.\nСокращайтесь, мышцы матки,\nвремя дорого: убийца\nвожделеет — взятки гладки —\nкровью с молоком упиться.\n+\nКто в подвале зачах?\nКто под землю несёт\nдевочку на сносях,\nраненную в живот?\nКто при свете свечи\nжизнью должен истечь?\n

In [229]:
records[162]

{'Text': '\nРевёт жена опальная,\nв правах поражена.\nИдёт война двуспальная,\nгражданская война.\nПомыты кухня, ванная\nслезами в три ручья.\nИдёт война диванная,\nтрещит по швам семья.\n+\nБудущему малышу,\nптицам небесным пища?\nСижу в уголке, пишу\nугольком пепелища\nродного. Роднее нет.\nМай. Четвёртое мая.\nВ окне кабинета свет:\nне выключила, убегая.\n+\nВсе страны, кроме одной.\nЧто же мы так бездомны?\nВстречаемся, мой родной,\nоколо пятой колонны\nбольшого театра войны —\nв Эривани, в Тифлисе,\nкак рыбе зонтик нужны\nмировой закулисе.\n+\nОт грохота, скрежета, лязга\nсжимается матка.\nКачает пустую коляску.\nЦе хлопчик? Дiвчатко.\nПоёт колыбельную дочке.\nПечаль непроглядна.\nВойна. Молока на сорочке\nрасплывающиеся пятна.\n+\nСвидетельств — терабайт,\nкак мой народ ушёл\nв искусственный офсайд.\nНо засчитали гол\nи торжествует зло,\nи стадион орёт.\nМне очень повезло\nуспеть на самолёт.\n+\nМеждународный язык —\nмладенческий лепет.\nСпит у груди призывник\nГолгофы. Колеблет\

In [230]:
counts = dict()
for recList in overlappingRecs:
    counts.setdefault(len(recList),0)
    counts[len(recList)] += 1

In [231]:
# create new list using only the earliest overlapping rec
repeatIdxs = set()
for recList in overlappingRecs:
    badRecs = decideRec(recList)
    badIdxs = [b['UniqueIndex'] for b in badRecs]
    for b in badIdxs:
        repeatIdxs.add(b)

In [232]:
cleanRecList = []
j = 0
for i, rec in enumerate(records):
    if rec['UniqueIndex'] not in repeatIdxs:
        rec['UniqueIndex'] = j
        cleanRecList.append(rec)
        j += 1
len(cleanRecList)

3289

In [234]:
cleanRecList[1000]

{'Text': '***\n \n– В Москве – невкусная клубника, бери водку!\n \nВ вытянутой трубке сна так тихо,\n \nЧто можно услышать\n \nНеуловимую чёрную лодку.\n \nСпрашиваю:\n \n– Это каяк?\n \nПтица с ветки:\n \n– Каяк, каяк. Что с тобою не так?\n \n– Со мною не так носовой платок,\n \nКаждое слово.\n \nНеба бесформенный носорог\n \nОживает в половине шестого.\n \nНаблюдаю за ним из окна.\n \nУ меня проблема только одна:\n \n«Скендербеу», «Хабнарфьордюр» и «Нымме Калью» проиграли свои матчи.\n \nИ три любимых, но непростых занятья:\n \nСмотреть на осину,\nСмотреть на берёзу,\nСмотреть на тополь.\n \nМошка умерла у меня на глазах,\nА что пережил ты?\n \n \n(интернет-журнал ”Literratura”)\n \n#выбор_Максима_Дрёмова',
 'Author': 'Денис Крюков',
 'Before or after': 'Before',
 'Source': 'metajournal',
 'Date posted': datetime.datetime(2019, 11, 14, 0, 0),
 'UniqueIndex': 1000}

In [233]:
cleanDf = pd.DataFrame.from_records(cleanRecList)

In [235]:
cleanDf.to_excel('../Excel_files/Full_Poem_Dataset_12-17.xlsx')

## AUTHOR DATA

In [7]:
authorInfo = pd.read_excel('../Excel_files/Thesis Authors.xlsx')
authorInfo.head()

,Number of Poems,Author,City,Country,Identification,Notes,Link to bio
0,3,Виген Аракелян,Razdan,Armenia,NaN,NaN,https://polutona.ru/?show=arakelian
1,1,Герман Лукомников,Baku,Azerbaijan,NaN,NaN,https://ru.wikipedia.org/wiki/%D0%9B%D1%83%D0%...
2,1,Александр Иличевский,Sumgait,Azerbaijan,NaN,NaN,https://ru.wikipedia.org/wiki/%D0%98%D0%BB%D0%...
3,7,Леонид Шваб,Babruysk,Belarus,NaN,NaN,http://www.litkarta.ru/world/israel/persons/sh...
4,6,Ирина Валерина,Babruysk,Belarus,NaN,NaN,NaN


In [9]:
authorInfo['Country'].value_counts()

Russia          370
Ukraine          86
Kazakhstan       26
Belarus          16
Uzbekistan        6
Moldova           5
Latvia            5
Estonia           4
Kyrgyzstan        2
Azerbaijan        2
Lithuania         1
Iran              1
Georgia           1
Tajikistan        1
Turkmenistan      1
USA               1
Armenia           1
Name: Country, dtype: int64

In [ ]:
value_counts